# NSI Stats

#### Colin Lindeman GIS Developer NiyamIT 2020-03-08
#### Python 3.7.x, Pandas, pyodbc

In [97]:
import pyodbc
import pandas as pd

#Suppress the scientific notation display...
pd.options.display.float_format = '{:.5f}'.format

# Get the data

## Create NSI 1 dataframe...

In [98]:
#DESKTOP-9JG5V4V\HAZUSPLUSSRVR
connectString = "Driver={SQL Server}; Server=SABRE-PC\HAZUSPLUSSRVR; Database=syHazus; UID=; PWD="
conn = pyodbc.connect(connectString, autocommit=False)

query_HI = "SELECT\
            [EqBldgTypeId],\
            [EqDesignLevelId],\
            [AreaSqft],\
            [ValStruct]\
            FROM [HI].[dbo].[tsNsiGbs]"

query_CA = "SELECT * FROM [CA].[dbo].[tsNsiGbs]"

dfNSI1 = pd.read_sql(query_HI, conn)

#make all columsn lowercase...
dfNSI1.columns = dfNSI1.columns.str.lower()

## Create NSI 2 dataframe...

In [99]:
csvPath = r"E:\Projects\NSI_20200200\data\HI15testoutput.csv"

dfNSI2 = pd.read_csv(csvPath)

#make all columsn lowercase...
dfNSI2.columns = dfNSI2.columns.str.lower()

#filter out unecessary fields...
dfNSI2 = dfNSI2[['eqbldgtypeid','eqdesignlevelid','areasqft','valstruct']]

### Get eqBldgType name from Id and assign...

In [100]:
query_eqBldgType = "SELECT \
                    [eqBldgType],\
                    [DisplayOrder]\
                    FROM [Hazus_model].[dbo].[eqclBldgType]"

df_eqBldgType = pd.read_sql(query_eqBldgType, conn)

eqBldgTypeDict = df_eqBldgType.to_dict('records')

def eqbldgtypeFunction(eqBldgTypeID):
    for item in eqBldgTypeDict:
        if eqBldgTypeID == item['DisplayOrder']:
            return item['eqBldgType']
    else:
        return -9

dfNSI1['eqbldgtype'] = dfNSI1.apply(lambda x: eqbldgtypeFunction(x['eqbldgtypeid']), axis=1)

dfNSI2['eqbldgtype'] = dfNSI2.apply(lambda x: eqbldgtypeFunction(x['eqbldgtypeid']), axis=1)

# Transform the data

## Record Counts...

In [101]:
print(f"NSI 1: {len(dfNSI1.index)} records")
print(f"NSI 2: {len(dfNSI2.index)} records")

NSI 1: 368627 records
NSI 2: 344996 records


## designlevel Counts...

In [102]:
dfNSI1DLCount = dfNSI1.groupby('eqdesignlevelid').count()
dfNSI1DLCount = dfNSI1DLCount[['areasqft']]
dfNSI1DLCount = dfNSI1DLCount.rename(columns={'areasqft':'nsi1_count'})

dfNSI2DLCount = dfNSI2.groupby('eqdesignlevelid').count()
dfNSI2DLCount = dfNSI2DLCount[['areasqft']]
dfNSI2DLCount = dfNSI2DLCount.rename(columns={'areasqft':'nsi2_count'})

In [103]:
#outer join the two dataframes...
dfDesignLevelCount = pd.merge(dfNSI1DLCount, dfNSI2DLCount, on='eqdesignlevelid', how='outer')
#subtract nsi 1 from 2 to see change...
dfDesignLevelCount['count_diff'] = dfDesignLevelCount['nsi2_count'] - dfDesignLevelCount['nsi1_count']
dfDesignLevelCount

,nsi1_count,nsi2_count,count_diff
eqdesignlevelid,,,
1,158102,62960,-95142
2,170578,222093,51515
3,17099,16697,-402
4,22848,43246,20398


## designlevel Sums...

In [104]:
dfNSI1DLSum = dfNSI1.groupby('eqdesignlevelid').sum()
dfNSI1DLSum = dfNSI1DLSum[['areasqft', 'valstruct']]
dfNSI1DLSum = dfNSI1DLSum.rename(columns={'areasqft':'nsi1_areasqft', 'valstruct':'nsi1_valstruct'})

dfNSI2DLSum = dfNSI2.groupby('eqdesignlevelid').sum()
dfNSI2DLSum = dfNSI2DLSum[['areasqft', 'valstruct']]
dfNSI2DLSum = dfNSI2DLSum.rename(columns={'areasqft':'nsi2_areasqft', 'valstruct':'nsi2_valstruct'})

In [105]:
#outer join the two dataframes...
dfDesignLevelSum = pd.merge(dfNSI1DLSum, dfNSI2DLSum, on='eqdesignlevelid', how='outer')
#subtract nsi 1 from 2 to see change...
dfDesignLevelSum['areasqft_diff'] = dfDesignLevelSum['nsi2_areasqft'] - dfDesignLevelSum['nsi1_areasqft']
dfDesignLevelSum['valstruct_diff'] = dfDesignLevelSum['nsi2_valstruct'] - dfDesignLevelSum['nsi1_valstruct']
dfDesignLevelSum

,nsi1_areasqft,nsi1_valstruct,nsi2_areasqft,nsi2_valstruct,areasqft_diff,valstruct_diff
eqdesignlevelid,,,,,,
1,390530235.58788,78818659904.28041,649471308.98525,71871669936.76405,258941073.39737,-6946989967.51636
2,428007095.83820,83212441704.45151,397371612.56393,56096767135.71916,-30635483.27427,-27115674568.73235
3,37595934.32976,7314049595.72302,35985791.61697,5106417901.95381,-1610142.71280,-2207631693.76921
4,69711485.70561,14135560969.00436,103492545.39949,15753795706.19920,33781059.69388,1618234737.19483


## bldgtype Counts...

In [106]:
dfNSI1BTCount = dfNSI1.groupby('eqbldgtype').count()
dfNSI1BTCount = dfNSI1BTCount[['eqbldgtypeid']]
dfNSI1BTCount = dfNSI1BTCount.rename(columns={'eqbldgtypeid':'NSI1_Count'})

dfNSI2BTCount = dfNSI2.groupby('eqbldgtype').count()
dfNSI2BTCount = dfNSI2BTCount[['eqbldgtypeid']]
dfNSI2BTCount = dfNSI2BTCount.rename(columns={'eqbldgtypeid':'NSI1_Count'})

In [107]:
#outer join the two dataframes...
dfBldgTypeCount = pd.merge(dfNSI1BTCount, dfNSI2BTCount, on='eqbldgtype', how='outer')
dfBldgTypeCount

,NSI1_Count_x,NSI1_Count_y
eqbldgtype,,
C1H,85.00000,nan
C1L,1791.00000,11399.00000
C1M,17.00000,1.00000
C2H,88.00000,nan
C2L,167229.00000,27468.00000
C2M,48.00000,503.00000
C3H,9.00000,nan
C3L,893.00000,nan
C3M,11.00000,9.00000


## bldgtype Sums...

In [108]:
dfNSI1BTSum = dfNSI1.groupby('eqbldgtype').sum()
dfNSI1BTSum = dfNSI1BTSum[['areasqft', 'valstruct']]
dfNSI1BTSum = dfNSI1BTSum.rename(columns={'areasqft':'nsi1_areasqft', 'valstruct':'nsi1_valstruct'})

dfNSI2BTSum = dfNSI2.groupby('eqbldgtype').sum()
dfNSI2BTSum = dfNSI2BTSum[['areasqft', 'valstruct']]
dfNSI2BTSum = dfNSI2BTSum.rename(columns={'areasqft':'nsi1_areasqft', 'valstruct':'nsi1_valstruct'})

In [109]:
#outer join the two dataframes...
dfBldgtypeSum = pd.merge(dfNSI1BTSum, dfNSI2BTSum, on='eqbldgtype', how='outer')
dfBldgtypeSum

,nsi1_areasqft_x,nsi1_valstruct_x,nsi1_areasqft_y,nsi1_valstruct_y
eqbldgtype,,,,
C1H,5936862.06428,1244939999.99958,nan,nan
C1L,23026833.40728,4990903492.71749,117207697.58750,7424135460.44215
C1M,1034024.76834,214755959.49655,776.25614,185244.12048
C2H,7143116.87172,1511818172.35898,nan,nan
C2L,366797067.31528,73972178109.29488,78894302.23244,10737573927.51524
C2M,2919599.32651,606373868.36435,4299591.97167,548332551.07687
C3H,547425.00686,113694999.99917,nan,nan
C3L,4950485.93400,973862962.85387,nan,nan
C3M,669074.88417,138959979.74827,207923.08193,40034309.08936


# Export the data

## Export to csv...

In [110]:
outputXlsx = r'E:\Projects\NSI_20200200\documentation\HI_NSI_Stats.xlsx'

with pd.ExcelWriter(outputXlsx) as writer:
    dfDesignLevelCount.to_excel(writer, sheet_name='dfDesignLevelCount')
    dfDesignLevelSum.to_excel(writer, sheet_name='dfDesignLevelSum')
    dfBldgTypeCount.to_excel(writer, sheet_name='dfBldgTypeCount')
    dfBldgtypeSum.to_excel(writer, sheet_name='dfBldgtypeSum')